In [273]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
import numpy as np
import json

In [274]:
# Define hyperparameters
input_vocab_size = 114 # Vocabulary size for the source language
output_vocab_size = 80 # Vocabulary size for the target language
embedding_dim = 100  # Dimension of the GloVe embeddings
hidden_units = 32 # Number of units in LSTM layers
sequence_length = 35 # Maximum sequence length
batch_size = 32 # Batch size
num_epochs = 200 # Number of training epochs

In [275]:
glove_path = 'GloVe/glove.6B.100d.txt'  # Adjust the path to your downloaded GloVe file
embedding_matrix = {}  # Create an empty dictionary to store the embeddings

In [276]:
with open(glove_path, 'r', encoding='utf-8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_matrix[word] = coefs #(coeff == Vectors)

In [277]:
# Create an embedding matrix for the source and target languages
source_embedding_matrix = np.zeros((input_vocab_size, embedding_dim))
target_embedding_matrix = np.zeros((output_vocab_size, embedding_dim))

In [278]:
##load the dataset
with open('card.json', 'r') as json_file:
    dataset = json.load(json_file)

In [279]:
#mapping input and output sequences to integers 
#input sequence = source_tokenizer
#output sequence = target_tokenizer

index_s = 1
index_t = 1
source_tokenizer = {}
target_tokenizer = {}


for i in dataset:
    question_toks = i["question_toks"]
    # print(query_toks)
    for j in question_toks:
        # print(j)
        
        if j not in source_tokenizer:
            source_tokenizer[j] = index_s
            index_s += 1
# print((frequency))
for i in dataset:
    query_toks = i["query_toks"]
    # print(query_toks)
    for j in query_toks:
        # print(j)
        
        if j not in target_tokenizer:
            target_tokenizer[j] = index_t
            index_t += 1

In [280]:
for word, i in source_tokenizer.items():
    embedding_vector = embedding_matrix.get(word)
    if embedding_vector is not None:
        source_embedding_matrix[i] = embedding_vector

for word, i in target_tokenizer.items():
    embedding_vector = embedding_matrix.get(word)
    if embedding_vector is not None:
        target_embedding_matrix[i] = embedding_vector

In [281]:
question_tokens = [example["question_toks"] for example in dataset]
query_tokens = [example["query_toks"] for example in dataset]

encoder_input_data_nopad = [[source_tokenizer[word] for word in sentence] for sentence in question_tokens]


decoder_input_data_nopad = [[target_tokenizer[word] for word in sentence] for sentence in query_tokens]


In [282]:
# Pad the sequences to a consistent length
def pad_sequences(sequences, max_length):
    padded_sequences = []
    for sequence in sequences:
        if len(sequence) < max_length:
            padded_sequence = sequence + [0] * (max_length - len(sequence))
        else:
            padded_sequence = sequence[:max_length]
        padded_sequences.append(padded_sequence)
    return padded_sequences

# Pad the encoder and decoder inputs
encoder_input_data = pad_sequences(encoder_input_data_nopad, sequence_length)
decoder_input_data = pad_sequences(decoder_input_data_nopad, sequence_length)
encoder_input_data = np.array(encoder_input_data)
decoder_input_data = np.array(decoder_input_data)

In [283]:
decoder_input_data

array([[ 1,  2,  3, ...,  0,  0,  0],
       [ 1,  2,  3, ...,  0,  0,  0],
       [ 1,  8,  9, ...,  0,  0,  0],
       ...,
       [ 1, 71,  6, ...,  0,  0,  0],
       [ 1,  8,  9, ...,  0,  0,  0],
       [ 1,  8,  9, ...,  0,  0,  0]])

In [284]:
#decoder_target_data
target_vocab = {
    0 : 0
}

decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]
decoder_target_data[:, -1] = target_vocab[0]

In [285]:
# Define the encoder
encoder_inputs = tf.keras.layers.Input(shape=(sequence_length,))
encoder_embedding = Embedding(input_vocab_size, embedding_dim, weights=[source_embedding_matrix], trainable=False)(encoder_inputs)
encoder_lstm = LSTM(hidden_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

In [286]:
# Define the decoder
decoder_inputs = tf.keras.layers.Input(shape=(sequence_length,))
decoder_embedding = Embedding(output_vocab_size, embedding_dim, weights=[target_embedding_matrix], trainable=False)(decoder_inputs)
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(output_vocab_size, activation='softmax')
output = decoder_dense(decoder_outputs)

In [287]:
# Build and compile the model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [288]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_15 (InputLayer)       [(None, 35)]                 0         []                            
                                                                                                  
 input_16 (InputLayer)       [(None, 35)]                 0         []                            
                                                                                                  
 embedding_14 (Embedding)    (None, 35, 100)              11400     ['input_15[0][0]']            
                                                                                                  
 embedding_15 (Embedding)    (None, 35, 100)              8000      ['input_16[0][0]']            
                                                                                            

In [289]:
print(decoder_target_data.shape)
print(encoder_input_data.shape)
print(decoder_input_data.shape)

(80, 35)
(80, 35)
(80, 35)


In [290]:
# Train the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=num_epochs, validation_split=0.2)

Epoch 1/200
2/2 [==============================] - 9s 2s/step - loss: 4.3970 - val_loss: 4.3620
Epoch 2/200
2/2 [==============================] - 0s 65ms/step - loss: 4.3650 - val_loss: 4.3315
Epoch 3/200
2/2 [==============================] - 0s 64ms/step - loss: 4.3328 - val_loss: 4.3000
Epoch 4/200
2/2 [==============================] - 0s 110ms/step - loss: 4.2996 - val_loss: 4.2657
Epoch 5/200
2/2 [==============================] - 0s 90ms/step - loss: 4.2641 - val_loss: 4.2269
Epoch 6/200
2/2 [==============================] - 0s 95ms/step - loss: 4.2233 - val_loss: 4.1817
Epoch 7/200
2/2 [==============================] - 0s 56ms/step - loss: 4.1766 - val_loss: 4.1268
Epoch 8/200
2/2 [==============================] - 0s 58ms/step - loss: 4.1213 - val_loss: 4.0586
Epoch 9/200
2/2 [==============================] - 0s 100ms/step - loss: 4.0529 - val_loss: 3.9724
Epoch 10/200
2/2 [==============================] - 0s 108ms/step - loss: 3.9707 - val_loss: 3.8628
Epoch 11/200
2/2 [

2/2 [==============================] - 0s 71ms/step - loss: 1.4912 - val_loss: 1.3220
Epoch 84/200
2/2 [==============================] - 0s 92ms/step - loss: 1.4839 - val_loss: 1.3204
Epoch 85/200
2/2 [==============================] - 0s 141ms/step - loss: 1.4726 - val_loss: 1.3215
Epoch 86/200
2/2 [==============================] - 0s 103ms/step - loss: 1.4652 - val_loss: 1.3216
Epoch 87/200
2/2 [==============================] - 0s 95ms/step - loss: 1.4564 - val_loss: 1.3226
Epoch 88/200
2/2 [==============================] - 0s 64ms/step - loss: 1.4492 - val_loss: 1.3216
Epoch 89/200
2/2 [==============================] - 0s 63ms/step - loss: 1.4409 - val_loss: 1.3185
Epoch 90/200
2/2 [==============================] - 0s 68ms/step - loss: 1.4336 - val_loss: 1.3155
Epoch 91/200
2/2 [==============================] - 0s 239ms/step - loss: 1.4251 - val_loss: 1.3154
Epoch 92/200
2/2 [==============================] - 0s 56ms/step - loss: 1.4167 - val_loss: 1.3148
Epoch 93/200
2/2 [==

2/2 [==============================] - 0s 93ms/step - loss: 1.0731 - val_loss: 1.3069
Epoch 166/200
2/2 [==============================] - 0s 63ms/step - loss: 1.0515 - val_loss: 1.2873
Epoch 167/200
2/2 [==============================] - 0s 65ms/step - loss: 1.0604 - val_loss: 1.2896
Epoch 168/200
2/2 [==============================] - 0s 63ms/step - loss: 1.0509 - val_loss: 1.3021
Epoch 169/200
2/2 [==============================] - 0s 59ms/step - loss: 1.0422 - val_loss: 1.2878
Epoch 170/200
2/2 [==============================] - 0s 53ms/step - loss: 1.0406 - val_loss: 1.2911
Epoch 171/200
2/2 [==============================] - 0s 63ms/step - loss: 1.0348 - val_loss: 1.3108
Epoch 172/200
2/2 [==============================] - 0s 64ms/step - loss: 1.0320 - val_loss: 1.3007
Epoch 173/200
2/2 [==============================] - 0s 86ms/step - loss: 1.0225 - val_loss: 1.2855
Epoch 174/200
2/2 [==============================] - 0s 67ms/step - loss: 1.0235 - val_loss: 1.2880
Epoch 175/200
